# 論文輪読
那須野薫(Kaoru Nasuno)/ 東京大学松尾研究室(Matsuo Lab., the University of Tokyo)

## メタ情報
タイトル：Teaching Machines to Read and Comprehend   
著者所属：Google DeepMind, University of Oxford  
公開：2015年6月  
リンク： [arXiv](http://arxiv.org/abs/1506.03340)  
ページ数：13   



## Abstract

自然言語文書を読むように機械を学習させることは、捉えどころのない課題である。  
機械による読解システムは提示された文書の内容に関する問題への回答能力に基づいて評価されるが、  
しかし、これまで、大規模な訓練およびテストデータセットはこの類の評価には用意されていなかった。  
この論文では、我々は大規模な教師あり学習用の文書読解データを提供し、このボトルネックを解消する新しい方法論を定義する。  
これによって、
言語構造に関する最低限の事前知識を用いることで、  
実際の文書を読んで難しい問題に回答することを学習するdeep neural networksに基づいたアテンションの類を開発できるようになる。




## 選択理由
#### DeepMind, Attention, RNNというキーワード

#### 自然言語処理の特に、モデルの知識獲得に関する内容






## 感想
ふーん、という感じ。  
結果に、強い意外感はない。  
タイトル読んだ段階でのイメージとやや違った。  
Unsupervised でやる方法を提案して欲しかった。  



## 目次
- Introduction
- Supervised training data for reading comprehension
- Models
- Empirical Evaluation











## 1. Introduction
bag-of-wordsによる情報検索アルゴリズムから文書読解ができる機械へのみちのりの進展はゆっくりである。  

教師あり自然言語読解に利用できるデータを作るのは、難しいことが示されれているが、  
some reasearchersは合成物語とクエリの生成を探索する研究している。  

本稿では、教師あり学習用の自然言語読解データセットの構築する新しい方法論を導入することで、  
現実の自然言語訓練データが不足しているという課題を、我々は直接扱おうとしている。

我々は、新しい読解用の深層学習モデルを構築し、新しいコーパスの有効性を示す。


## 2. Supervised training data for reading comprehension
読解タスクは自然と、教師あり学習問題の定式化となろう。
Specifically we seek to estimate the conditional probability p(a|c, q), where c is a context document, q a query relating to that document, and a the answer to that query.
特に、$p(a|c, q)$という条件付き確率を推定する問題を解くことになる（qがクエリ(質問)、aが回答、cが文脈つまり文書）。

こうしたアプローチは、大規模な訓練用のコーパス（文書-質問-回答のトリプレット）を必要とするが、  
これまでそんなコーパスは数百程度の事例に限られており、したがって、ほとんど検証ようにしか使えなかった。

ここでは、現実世界の大規模な教師あり学習の読解モデル用の訓練データを構築する方法論を提案する。  
CNNの11万記事とDailyMailの22万記事を集めた。  
記事の要約がリスト形式で存在するが、それらは記事の抽象であり、抜き出しではないため、この実験に利用できる。    
文書-質問-回答のトリプレットを各要約を先行文献の方法に従い、質問文に書き換えることで、構築した。   
結果として、1M程度のデータを得た。  


<img src='files/img/Read_and_Comprehend01.png' width="900px"/>

### 2-1. Qualitative Analysis of Answer Difficulty
タスクが難しいってことを言いたい。  

人間に解かせた。    
Simple：文法や意味的な複雑性がないもの。  
Lexical：語彙的な一般化が必要なもの。  
Coref：同一指示の解決が必要なもの。  
Cref/Lex：Lexcalで、かつ、Corefなもの。  
Complex：因果関係処理や、他の複雑な推論が必要なもの。  
Unanswerable：回答不能。  

30%はComplexで、10%は回答不能だから、めっちゃ難しい。


### 2-2. Entity replacement and permutation
<img src='files/img/Read_and_Comprehend02.png' width="900px"/>

本研究の目的は、読解能力を評価したいのであって、世界知識や単語の共起を評価したいのではない。  
例)
ハイテクなブラは乳Xを防ぐ  
SaccharinはXに効く  
魚の脂は前立腺Xに効く  
X=ガン  
こうした事例は、知識や単語の共起で解けてしまうので、問題として不適。  


そうした悪い解決方法に陥らないようにするために、  下記の方法で、匿名化とランダム化をおこなった。
1. 単語の同一指示処理  
2. 同一単語を同一IDで置き換え、匿名化  
3. 文書のロードの度にID名をランダムに変える。  



## 3. Models
ベースライン2つ、  
シンボリックマッチング2つ、  
深層学習3つ。

### 3-0. ベースライン
#### Maximum frequency
当該文脈における単語の最大出現頻度をもとに回答。  
#### Exclusive frequency
当該文脈における単語の最大出現頻度をもとに回答するが、質問文には含まれていないものを回答。  

### 3-1. Symbolic Matching Models
伝統的には自然言語処理モデルをパイプでつなげて、問題回答が行われていた。  
つまり、言語アノテーション、構造化された単語の知識や意味への分解とか。
そうした手法をいくつか。
#### Frame-semantic Parsing
<img src='files/img/Read_and_Comprehend03.png' width="900px"/>
「誰が誰に何をしたか」を特定しようという手法。  



#### Word Distance Benchmark
質問文におけるすべての単語について、もっとも近くに位置する文脈文の単語との距離の足しあわたものをスコアとする。  
ちょっと、よく分からない。
$$
p(a | d, q) \propto exp(W(a)g(d, q)), s.t. a \in d,
$$
ここでは、
$W(a)0$は$W$の$a$行目、  
関数$g(d, q)$は文書と質問文のペアを表現するベクトルを返す。  

where W (a) indexes row a of weight matrix W and through a slight abuse of notation word types double as indexes. Note that we do not privilege entities or variables, the model must learn to differentiate these in the input sequence. The function g(d,q) returns a vector embedding of a document and query pair.


### 3-2. Neural Network Models
#### The Deep LSTM Reader
<img src='files/img/Read_and_Comprehend06.png' width="900px"/>
<img src='files/img/Read_and_Comprehend07.png' width="600px"/>
2層のLSTM。  
数式のまま。  
わかり易い。  
#### The Attentive Reader
<img src='files/img/Read_and_Comprehend04.png' width="600px"/>
<img src='files/img/Read_and_Comprehend08.png' width="300px"/>
bi-directional LSTMを使った、ただのattentionモデル。  
文書(左)の方は、attentionするため、すべての隠れ層を$r$への入力とする。  
質問文(右)の方は、各方向の最後の隠れ層を$u$への入力とする。  
わかり易い。

<img src='files/img/Read_and_Comprehend09.png' width="300px"/>


#### The Impatient Reader
<img src='files/img/Read_and_Comprehend05.png' width="600px"/>
<img src='files/img/Read_and_Comprehend10.png' width="600px"/>
<img src='files/img/Read_and_Comprehend11.png' width="300px"/>

わかりにくい。  
The Attentive Readerをちょっとかえたもの。  
質問文の新しい単語が読まれるたびに、文書を読み返している。  
$r(i)$が$|q|$分だけある。
$r(i)$はrecurrentになっている。





## 4. Empirical Evaluation
目的
1. 幅広いモデルを適用して、このタスクが難しいということを確立させること。  
2. ニューラルモデルとパースベースの手法を比較すること。  
3. ニューラルモデルの中で、どの部分が予測に寄与しているのか明らかにすること。特にアテンションが。  


<img src='files/img/Read_and_Comprehend12.png' width="900px"/>
値はAccuracy。  
60%が正しく回答できており、  
これは、解けないか、複雑な推論を用いない問題はだいたい回答できていることを意味する。
<img src='files/img/Read_and_Comprehend13.png' width="900px"/>


(Appendixに、他のattentionの事例あり。)